Import thư viện

In [248]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
import category_encoders as ce

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Đọc dữ liệu tập train, test

In [249]:
df = pd.read_csv("train.csv")
train_labels = df['Risk']
train_data = df.drop('Risk',axis = 'columns')
train_data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,47,male,2,free,NaN,little,8335,36,car
1,38,male,2,own,little,NaN,804,12,radio/TV
2,28,male,2,own,little,little,5371,36,furniture/equipment
3,29,female,0,own,NaN,moderate,3990,36,domestic appliances
4,24,female,2,own,NaN,moderate,8487,48,car


In [250]:
test_df = pd.read_csv("test.csv")
test_labels = test_df['Risk']
test_data = test_df.drop('Risk', axis='columns')
test_data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,36,male,2,own,rich,moderate,1913,18,business
1,34,male,3,own,little,moderate,1860,12,car
2,48,male,2,own,little,little,1024,24,radio/TV
3,31,male,2,own,little,little,3104,18,business
4,23,male,1,own,quite rich,moderate,2520,27,radio/TV
...,...,...,...,...,...,...,...,...,...
195,53,male,2,free,little,little,7119,48,furniture/equipment
196,42,male,2,own,NaN,moderate,2427,18,business
197,25,male,2,own,little,NaN,1262,12,radio/TV
198,31,male,2,own,NaN,little,6350,30,furniture/equipment


In [251]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               800 non-null    int64 
 1   Sex               800 non-null    object
 2   Job               800 non-null    int64 
 3   Housing           800 non-null    object
 4   Saving accounts   656 non-null    object
 5   Checking account  484 non-null    object
 6   Credit amount     800 non-null    int64 
 7   Duration          800 non-null    int64 
 8   Purpose           800 non-null    object
dtypes: int64(4), object(5)
memory usage: 56.4+ KB


Thay thể giá trị missing là 'nan'

In [252]:
missing_value_cols = ['Saving accounts','Checking account']

In [253]:
train_data[missing_value_cols] = train_data[missing_value_cols].replace(np.nan,'nan')
test_data[missing_value_cols] = test_data[missing_value_cols].replace(np.nan,'nan')

In [254]:
train_data.dtypes

Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
dtype: object

In [255]:
# for value in train_data['Credit amount']:
#     if value > 10000: value = 10000


# train_data['Credit amount'] = train_data['Credit amount'].map(lambda x: 10000 if x > 10000 else x)
# train_data['Credit amount'] = np.log(train_data['Credit amount']+1)

# print(train_data['Credit amount'].unique())

Vì tỉ lệ rủi ro không tăng theo tuổi, nghề nghiệp nên từ dạng numeric đổi về dạng object

In [256]:
def age_transform(value):
    if value <= 29: value = 'Young'
    elif value <= 40 : value = 'Young_Adult'
    elif value <55: value = 'Adult'
    elif value <80: value = 'Elder'
    else:
        value = 'Diff'
    return value

train_data['Age'] = train_data['Age'].map(age_transform)
test_data['Age'] = test_data['Age'].map(age_transform)

In [257]:
# def job_transform(value):
#     if value == 0 : value = 'No rent'
#     elif value == 1 : value = 'None skill - No rent'
#     elif value == 2: value = 'Skill - Rent'
#     elif value == 3 : value = 'High skill'
#     else: value = 'None'
#     return value

# train_data['Job'] = train_data['Job'].map(job_transform)
# test_data['Job'] = test_data['Job'].map(job_transform)

In [258]:
numeric_cols = ['Job','Credit amount','Duration']
categorical_cols = ['Sex','Housing','Saving accounts','Checking account','Purpose', 'Age']

Encode lại các cột object theo tỉ lệ rủi ro trên tập train

In [259]:
encoder = ce.TargetEncoder(categorical_cols)
encoder.fit(X=train_data, y= train_labels)
train_data= encoder.transform(train_data)
test_data = encoder.transform(test_data)

c:\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
c:\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [260]:
train_data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0.256579,0.274047,2,0.406977,0.166667,0.479452,8335,36,0.309353
1,0.261745,0.274047,2,0.259649,0.364211,0.120253,804,12,0.213953
2,0.352740,0.274047,2,0.259649,0.364211,0.479452,5371,36,0.328859
3,0.352740,0.357430,0,0.259649,0.166667,0.392523,3990,36,0.300000
4,0.352740,0.357430,2,0.259649,0.166667,0.392523,8487,48,0.309353
...,...,...,...,...,...,...,...,...,...
795,0.352740,0.357430,2,0.259649,0.333333,0.120253,1237,6,0.213953
796,0.256579,0.274047,2,0.406977,0.364211,0.479452,1333,24,0.309353
797,0.256579,0.274047,2,0.259649,0.364211,0.120253,2515,18,0.328859
798,0.352740,0.357430,2,0.395833,0.166667,0.479452,8471,18,0.466667


In [261]:
for col in categorical_cols:
    print(col,train_data[col].unique())

Sex [0.27404719 0.35742972]
Housing [0.40697674 0.25964912 0.39583333]
Saving accounts [0.16666667 0.36421053 0.33333333 0.12195122 0.16      ]
Checking account [0.47945205 0.12025316 0.39252336 0.25490196]
Purpose [0.30935252 0.21395349 0.32885906 0.3        0.46666667 0.38888889
 0.33311024 0.32911392]
Age [0.25657895 0.26174497 0.35273973 0.34482759]


In [262]:
# train_data['Housing'] = train_data['Housing'].apply(lambda x: 0 if x == 'free' else (2 if x == 'own' else 1))
# train_data['Saving accounts'] = train_data['Saving accounts'].apply(lambda x: 2 if x == 'nan' 
#                                                                     else(0 if x == 'little' 
#                                                                     else(1 if x == 'moderate'
#                                                                     else(3 if x == 'quite rich' 
#                                                                     else 4))))
                                                                    
# train_data['Checking account'] = train_data['Checking account'].apply(lambda x: 2 if x == 'nan' 
#                                                                     else(0 if x == 'little' 
#                                                                     else(1 if x == 'moderate'
#                                                                     else(3 if x == 'quite rich' 
#                                                                     else 4))))

In [263]:
# test_data['Housing'] = test_data['Housing'].apply(lambda x: 0 if x == 'free' else (2 if x == 'own' else 1))
# test_data['Saving accounts'] = test_data['Saving accounts'].apply(lambda x: 0 if x == 'nan' 
#                                                                     else(1 if x == 'little' 
#                                                                     else(2 if x == 'moderate'
#                                                                     else(3 if x == 'quite rich' 
#                                                                     else 4))))
                                                                    
# test_data['Checking account'] = test_data['Checking account'].apply(lambda x: 0 if x == 'nan' 
#                                                                     else(1 if x == 'little' 
#                                                                     else(2 if x == 'moderate'
#                                                                     else(3 if x == 'quite rich' 
#                                                                     else 4))))

In [264]:
# train_data['Purpose'] = train_data['Purpose'].apply(lambda x: 0 if x == 'education' 
#                                                                     else(2 if x == 'radio/TV'                                                             
#                                                                     else 1))
# test_data['Purpose'] = test_data['Purpose'].apply(lambda x: 0 if x == 'education' 
#                                                                 else(2 if x == 'radio/TV'                                                             
#                                                                 else 1))

In [265]:
# train_data['Sex'] = train_data['Sex'].apply(lambda x: 0 if x == 'female' else 1)
# test_data['Sex'] = test_data['Sex'].apply(lambda x: 0 if x == 'female' else 1)


Scale lại bộ dữ liệu và sử dụng smote để sinh ra thêm dữ liệu

In [266]:
scaler = MinMaxScaler().fit(train_data[numeric_cols])
train_data[numeric_cols] = scaler.transform(train_data[numeric_cols])
train_data, train_labels = SMOTE().fit_resample(train_data, train_labels)
train_data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0.256579,0.274047,0.666667,0.406977,0.166667,0.479452,0.514328,0.571429,0.309353
1,0.261745,0.274047,0.666667,0.259649,0.364211,0.120253,0.033697,0.142857,0.213953
2,0.352740,0.274047,0.666667,0.259649,0.364211,0.479452,0.325164,0.571429,0.328859
3,0.352740,0.357430,0.000000,0.259649,0.166667,0.392523,0.237029,0.571429,0.300000
4,0.352740,0.357430,0.666667,0.259649,0.166667,0.392523,0.524028,0.785714,0.309353
...,...,...,...,...,...,...,...,...,...
1115,0.352740,0.337929,0.666667,0.291498,0.318012,0.120253,0.485560,0.571429,0.313974
1116,0.306929,0.315451,0.666667,0.259649,0.364211,0.392523,0.033308,0.116256,0.359081
1117,0.326403,0.274047,0.666667,0.259649,0.364211,0.479452,0.238225,0.707916,0.237520
1118,0.352740,0.357430,0.666667,0.363389,0.333333,0.392523,0.144930,0.209191,0.309353


In [267]:

test_data[numeric_cols] = scaler.transform(test_data[numeric_cols])


In [268]:
# feat_weight = np.array([1,1,0.5,1,2,2,2,1.5,1.5])
# i = 0
# for col in train_data.columns:
#     train_data[col] =  train_data[col]*feat_weight[i]
#     test_data[col] =  test_data[col]*feat_weight[i]
#     i+=1


In [269]:
train_data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0.256579,0.274047,0.666667,0.406977,0.166667,0.479452,0.514328,0.571429,0.309353
1,0.261745,0.274047,0.666667,0.259649,0.364211,0.120253,0.033697,0.142857,0.213953
2,0.352740,0.274047,0.666667,0.259649,0.364211,0.479452,0.325164,0.571429,0.328859
3,0.352740,0.357430,0.000000,0.259649,0.166667,0.392523,0.237029,0.571429,0.300000
4,0.352740,0.357430,0.666667,0.259649,0.166667,0.392523,0.524028,0.785714,0.309353
...,...,...,...,...,...,...,...,...,...
1115,0.352740,0.337929,0.666667,0.291498,0.318012,0.120253,0.485560,0.571429,0.313974
1116,0.306929,0.315451,0.666667,0.259649,0.364211,0.392523,0.033308,0.116256,0.359081
1117,0.326403,0.274047,0.666667,0.259649,0.364211,0.479452,0.238225,0.707916,0.237520
1118,0.352740,0.357430,0.666667,0.363389,0.333333,0.392523,0.144930,0.209191,0.309353


In [270]:
test_data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0.261745,0.274047,0.666667,0.259649,0.121951,0.392523,0.104474,0.250000,0.329114
1,0.261745,0.274047,1.000000,0.259649,0.364211,0.392523,0.101091,0.142857,0.309353
2,0.256579,0.274047,0.666667,0.259649,0.364211,0.479452,0.047738,0.357143,0.213953
3,0.261745,0.274047,0.666667,0.259649,0.364211,0.479452,0.180484,0.250000,0.329114
4,0.352740,0.274047,0.333333,0.259649,0.160000,0.392523,0.143213,0.410714,0.213953
...,...,...,...,...,...,...,...,...,...
195,0.256579,0.274047,0.666667,0.406977,0.364211,0.479452,0.436722,0.785714,0.328859
196,0.256579,0.274047,0.666667,0.259649,0.166667,0.392523,0.137277,0.250000,0.329114
197,0.352740,0.274047,0.666667,0.259649,0.364211,0.120253,0.062927,0.142857,0.213953
198,0.261745,0.274047,0.666667,0.259649,0.166667,0.479452,0.387644,0.464286,0.328859


Thực hiện train model

In [271]:

gnb = GaussianNB()
gnb.fit(train_data,train_labels)

GaussianNB()

In [272]:

knn = KNeighborsClassifier(n_neighbors=7 ,weights='distance')
knn.fit(train_data, train_labels)


KNeighborsClassifier(n_neighbors=7, weights='distance')

Dự đoán và kết quả

In [273]:
predgnb = gnb.predict(test_data)
# print(predgnb,len(np.where(predgnb==1)[0]))
print(classification_report(y_pred=predgnb, y_true = test_labels))

              precision    recall  f1-score   support

           0       0.87      0.69      0.77       140
           1       0.51      0.75      0.61        60

    accuracy                           0.71       200
   macro avg       0.69      0.72      0.69       200
weighted avg       0.76      0.71      0.72       200



In [274]:
predknn = knn.predict(test_data)
# print(predknn,len(np.where(predknn==1)[0]))
print(classification_report(y_pred=predknn, y_true = test_labels))

              precision    recall  f1-score   support

           0       0.79      0.66      0.72       140
           1       0.43      0.58      0.49        60

    accuracy                           0.64       200
   macro avg       0.61      0.62      0.61       200
weighted avg       0.68      0.64      0.65       200

